In [ ]:
import numpy as np
import xarray as xr
from kalman_reconstruction import pipeline
from kalman_reconstruction import example_models
import matplotlib.pyplot as plt

Now there can be easily multiple random variables add and the Kalman_SEM performs on all of them

In [ ]:
data = example_models.Lorenz_63_xarray(dt=0.01, time_length=5, time_steps=None)
seed = np.random.default_rng(seed=2349832653).integers(0, 1e12, 1)
variance = 5
rng1 = np.random.default_rng(seed=seed)
rng2 = np.random.default_rng(seed=seed + 1)
rng3 = np.random.default_rng(seed=seed + 2)
rng4 = np.random.default_rng(seed=seed + 3)
pipeline.add_random_variable(
    data, var_name="z1", random_generator=rng1, variance=variance
)
pipeline.add_random_variable(
    data, var_name="z2", random_generator=rng2, variance=variance
)
pipeline.add_random_variable(
    data, var_name="z3", random_generator=rng3, variance=variance
)
pipeline.add_random_variable(
    data, var_name="z4", random_generator=rng4, variance=variance
)

In [ ]:
test = pipeline.run_Kalman_SEM_to_xarray(
    ds=data,
    state_variables=["x1", "x2", "x3"],
    random_variables=[],
    nb_iter_SEM=10,
    variance_obs_comp=0.0001,
)

for var in test.state_names:
    plt.plot(test.time, test.states.sel(state_names=var), label=var.values)
    plt.fill_between(
        test.time,
        (
            test.states.sel(state_names=var)
            - 1.96 * test.uncertainties.sel(state_names=var, state_names_copy=var)
        ),
        (
            test.states.sel(state_names=var)
            + 1.96 * test.uncertainties.sel(state_names=var, state_names_copy=var)
        ),
        label=var.values,
        alpha=0.25,
    )

plt.legend()
plt.xlim((0, 2))
plt.xlabel("time")
plt.ylabel("Values")
plt.title("Validation using all states")

100%|██████████| 10/10 [00:02<00:00,  3.74it/s]


Text(0.5, 1.0, 'Validation using all states')

In [ ]:
test = pipeline.run_Kalman_SEM_to_xarray(
    ds=data,
    state_variables=["x1", "x2", "x3"],
    random_variables=["z1", "z2"],
    nb_iter_SEM=10,
    variance_obs_comp=0.0001,
)

for var in test.state_names:
    plt.plot(test.time, test.states.sel(state_names=var), label=var.values)
    plt.fill_between(
        test.time,
        (
            test.states.sel(state_names=var)
            - 1.96 * test.uncertainties.sel(state_names=var, state_names_copy=var)
        ),
        (
            test.states.sel(state_names=var)
            + 1.96 * test.uncertainties.sel(state_names=var, state_names_copy=var)
        ),
        alpha=0.25,
    )

plt.legend()
plt.xlim((0, 2))
plt.xlabel("time")
plt.ylabel("Values")
plt.title("Using multiple random latent variables")

100%|██████████| 10/10 [00:03<00:00,  2.95it/s]


Text(0.5, 1.0, 'Using multiple random latent variables')